# Compile reindexation files

## Setup

### Libraries

In [1]:
# Import librairies
import os
import re

import numpy as np
import pandas as pd
import seaborn as sns
import warnings

In [2]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

### Paths

In [3]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "/data"
data_path_reindex = "/home/aurelie/ABES/labo-indexation-ai/data/reindexation/avril2023"

# Import data

In [4]:
# Filenames
dataset = "working_data_sans_dewey.pkl"
rameau_file = "./extraction/liste_concepts_rameau_sans_vedetteConstruites.csv"

In [5]:
# Import rameau list
ram = pd.read_csv(os.path.join(data_path, rameau_file), encoding="latin-1")
print("Nombre de concepts différents: ", ram.shape[0])
ram.head(10)

Nombre de concepts différents:  85876


,PPN,NOM
0,157992527,Kirp?n
1,110140494,Militaires artistes
2,028492161,Militaires romains
3,028521757,Militaires prussiens
4,029895561,Sa-skya-pa
5,031875459,Militaires réunionnais
6,032370083,Construction à l'épreuve de la sécheresse
7,032878117,Missionnaires suisses
8,034423982,Militaires ivoiriens
9,034686940,Outils à métaux


In [6]:
# Import dataset
df = pd.read_pickle(os.path.join(data_path, dataset))
df["PPN"] = df["PPN"].astype(str)
print("Dimension of the dataset: ", df.shape)
df.head()

Dimension of the dataset:  (154508, 9)


,PPN,TITRE,RESUME,RAMEAU,DEWEY,DESCR,presence_chaine_indexation,rameau_chaines_index,rameau_concepts
0,000002364,La culture pour vivre,Mort de la culture populaire en France. Mutati...,Culture populaire;Diffusion de la culture;Poli...,840,La culture pour vivre Mort de la culture popul...,False,"[Culture populaire, Diffusion de la culture, P...","[Culture populaire, Diffusion de la culture, P..."
1,000014877,"La nuit, le jour : essai psychanalytique sur l...","Discontinuité, latence, rétablissement d’une c...",Complexe de castration;Psychanalyse;Rêves,154.63,"La nuit, le jour : essai psychanalytique sur l...",False,"[Complexe de castration, Psychanalyse, Rêves]","[Complexe de castration, Psychanalyse, Rêves]"
2,000021857,"Ruptures, cultures","Il faut imaginer Robinson sur son île, au mome...",Culture,840,"Ruptures, cultures Il faut imaginer Robinson s...",False,[Culture],[Culture]
3,00002564X,La révolution structurale,"Mutations ou crises, les brusques accès de fiè...",Structuralisme,100,"La révolution structurale Mutations ou crises,...",False,[Structuralisme],[Structuralisme]
4,000026352,La Destruction du temple,"Oswald tire sur Kennedy. Jusque-là, c'est bon,...",Science-fiction américaine -- Traductions fran...,830,La Destruction du temple Oswald tire sur Kenne...,True,[Science-fiction américaine -- Traductions fra...,"[Science-fiction américaine, Traductions franç..."


In [7]:
# list files
file_list = os.listdir(data_path_reindex)
file_list

['100 NOTICES A INDEXER_AFE.xlsx',
 '100 NOTICES A INDEXER-MCR.xlsx',
 '100 NOTICES A INDEXER - JMF.xlsx',
 '100 NOTICES A INDEXER_LPL.xlsx',
 '100 NOTICES A INDEXER.xlsx',
 '100 NOTICES A INDEXER-LJZ.xlsx',
 '100_notices.xlsx',
 '150_notices.xlsx',
 'ppn_echantillon_150notices.csv',
 '100 NOTICES A INDEXER_MPD.xlsx',
 'ppn_echantillon_100notices_stratified_on_50_clusters.csv']

In [8]:
index_file_list = [
    '100 NOTICES A INDEXER_AFE.xlsx',
    '100 NOTICES A INDEXER-MCR.xlsx',
    '100 NOTICES A INDEXER - JMF.xlsx',
    '100 NOTICES A INDEXER_LPL.xlsx',
    '100 NOTICES A INDEXER-LJZ.xlsx',
    '100 NOTICES A INDEXER_MPD.xlsx']

In [9]:
# Check import for one file
file = '100 NOTICES A INDEXER.xlsx'
ind = pd.read_excel(
    os.path.join(data_path_reindex, file), 
    sheet_name="Notices à indexer", 
    engine="openpyxl", 
    converters={"PPN": lambda x: str(x)})
ind.head(3)

,N°ordre,PPN,TITRE,RESUME,INDEXATION UNIMARC,INDEXATION AFFICHAGE PUBLIC
0,79,308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'Etat ""fort"" à la...",NaN,NaN
1,58,3632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",NaN,NaN
2,59,47450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",NaN,NaN


## Check files

In [10]:
file = '100 NOTICES A INDEXER_AFE.xlsx'
ind = pd.read_excel(
    os.path.join(data_path_reindex, file), 
    sheet_name="Notices à indexer", 
    skiprows=lambda x: x == 1, 
    engine="openpyxl", 
    converters={"PPN": lambda x: str(x)})
ind.drop(ind[ind["PPN"].isna()].index, inplace=True) # delete first line corresponding to example
ind["INDEXATION AFFICHAGE PUBLIC"] = ind["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub('\\n$', '', x))
ind["INDEXATION AFFICHAGE PUBLIC"] = ind["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub('^\\n', '', x))
ind["INDEXATION AFFICHAGE PUBLIC"] = ind["INDEXATION AFFICHAGE PUBLIC"].replace("\\n", ';', regex=True)
ind["INDEXATION AFFICHAGE PUBLIC"] = ind["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub(';*$', '', x))
ind["INDEXATION AFFICHAGE PUBLIC"] = ind["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub('^;*', '', x))
ind.head(3)

,N°ordre,PPN,TITRE,RESUME,INDEXATION UNIMARC,INDEXATION AFFICHAGE PUBLIC
0,58,3632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France‎ -- 1958-.... (5e Répu...
1,59,47450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme
2,65,58296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",606$302736576X$aPhilosophie et sciences,Philosophie et sciences


In [11]:
def import_indexation(file):
    df = pd.read_excel(
        os.path.join(data_path_reindex, file), 
        sheet_name="Notices à indexer", 
        engine="openpyxl", 
        converters={"PPN": lambda x: str(x)})
    df.drop(df[df["PPN"].isna()].index, inplace=True) # delete first line corresponding to example
    if "INDEXATION AFFICHAGE PUBLIC" in df.columns:
        if any(pd.isnull(df["INDEXATION AFFICHAGE PUBLIC"])):
            df.loc[pd.isnull(df["INDEXATION AFFICHAGE PUBLIC"]), "INDEXATION AFFICHAGE PUBLIC"] = ''
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub('\\n*$', '', x)) 
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub('^\\n*', '', x))  
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: ';;'.join(x.splitlines())) 
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].replace("\\n", '', regex=True) 
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].replace("\u200e", '', regex=True)
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("^[\s*;\s*]", '', x))
        df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("[\s*;\s*]$", '', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Ecologie", 'Écologie', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Education", 'Éducation', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Econo", 'Écono', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Etablissement", 'Établissement', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Evaluation", 'Évaluation', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Evolution", 'Évolution', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Ecrivain", 'Écrivain', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Etude", 'Étude', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Energie", 'Énergie', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Ecoterrorisme", 'Écoterrorisme', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Ecole", 'École', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Etat", 'État', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Eolienne", 'Éolienne', x))    
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Equipe", 'Équipe', x))         
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Eco-anxiété", 'Éco-anxiété', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Epistémologie", 'Épistémologie', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Extémisme", 'Extrémisme', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Parimoine", 'Patrimoine', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Ralation", 'Relation', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Jusrisprudence", 'Jurisprudence', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Procéudure", 'Procédure', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("socialisme et écologie", 'Socialisme et écologie', x))
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("Vendrée", 'Vendée', x))    
        # df["INDEXATION AFFICHAGE PUBLIC"] = df["INDEXATION AFFICHAGE PUBLIC"].apply(lambda x: re.sub("$", '', x))    

    df["PPN"] = df["PPN"].astype(str)
    
    return df
    

In [12]:
ppn_to_check = []
for file in index_file_list:
    print("Dealing with file :", file)
    ind = import_indexation(file)
    check = []

    for ppn in ind["PPN"].astype(str):
        if any(df["PPN"] == ppn):
            #print("PPN matché à la ligne :", df.loc[df["PPN"] == ppn].index)
            check.append(True)
        else: 
            print("Pas de correspondance pour PPN ", ppn)
            check.append(False)
            ppn_to_check.append(ppn)

    print("Verification que tous les PPN matchent: ", all(check))

Dealing with file : 100 NOTICES A INDEXER_AFE.xlsx
Pas de correspondance pour PPN  308838
Pas de correspondance pour PPN  3632806
Pas de correspondance pour PPN  47450037
Pas de correspondance pour PPN  58296182
Pas de correspondance pour PPN  59911174
Pas de correspondance pour PPN  67313493
Pas de correspondance pour PPN  70072973
Pas de correspondance pour PPN  76503909
Pas de correspondance pour PPN  76986152
Pas de correspondance pour PPN  77463560
Pas de correspondance pour PPN  77880560
Pas de correspondance pour PPN  86077368
Verification que tous les PPN matchent:  False
Dealing with file : 100 NOTICES A INDEXER-MCR.xlsx
Pas de correspondance pour PPN  308838
Pas de correspondance pour PPN  3632806
Pas de correspondance pour PPN  47450037
Pas de correspondance pour PPN  58296182
Pas de correspondance pour PPN  59911174
Pas de correspondance pour PPN  67313493
Pas de correspondance pour PPN  70072973
Pas de correspondance pour PPN  76503909
Pas de correspondance pour PPN  76986

In [13]:
ppn_to_correct = list(set(ppn_to_check))
ppn_to_correct

['67313493',
 '70072973',
 '308838',
 '86077368',
 '58296182',
 '59911174',
 '47450037',
 '77463560',
 '3632806',
 '76986152',
 '76503909',
 '77880560']

## Merge files (par les PPN)

In [14]:
cols_to_merge = ["PPN", "INDEXATION UNIMARC", "INDEXATION AFFICHAGE PUBLIC"]

In [15]:
def merge_new_indexation(file1, file2, columns, on="PPN", how="outer"):
    if isinstance(file1, pd.DataFrame):
        df1 = file1
    else: 
        df1 = import_indexation(file1)
    if isinstance(file2, pd.DataFrame):
        df2 = file2
    else: 
        df2 = import_indexation(file2)
    df_merge = pd.merge(df1, df2[columns], on=on, how=how)
    return df_merge

In [16]:
# Check notices extraction 
index_file = '100 NOTICES A INDEXER.xlsx'
df = import_indexation(index_file)
print(df.shape)
df

(100, 6)


,N°ordre,PPN,TITRE,RESUME,INDEXATION UNIMARC,INDEXATION AFFICHAGE PUBLIC
0,79,308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'Etat ""fort"" à la...",NaN,
1,58,3632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",NaN,
2,59,47450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",NaN,
3,65,58296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod...",NaN,
4,80,59911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...,NaN,
...,...,...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ...",NaN,
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...,NaN,
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ...",NaN,
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...,NaN,


In [17]:
# Remove useless columns
df.drop(columns=["INDEXATION UNIMARC", "INDEXATION AFFICHAGE PUBLIC"], inplace=True)
df

,N°ordre,PPN,TITRE,RESUME
0,79,308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'Etat ""fort"" à la..."
1,58,3632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ..."
2,59,47450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon..."
3,65,58296182,L'homme devant l'incertain,"La quatrième de couverture indique : ""Réintrod..."
4,80,59911174,Imperator Caesar Augustus : recherches sur le ...,Ce dossier comporte quatre volumes qui aborden...
...,...,...,...,...
95,38,23690454X,Droit public et patrimoine : le rôle du Consei...,"la 4e de couv. indique : ""L'histoire du droit ..."
96,95,24155859X,Lars Fredrikson : [exposition Musée d'art mode...,Première grande monographie consacrée à l'œuvr...
97,52,25561280X,L'ère des soulèvements : émeutes et confinemen...,"Dès les années 1980, Michel Maffesoli se fait ..."
98,99,26117309X,Monde enchanté : chansons et imaginaires géogr...,Le monde est enchanté par certaines chansons. ...


In [18]:
# Merge notices indexation
index_columns = []
for file in index_file_list:
    name = file[-8:-5]
    print("Dealing with :", name)
    df = merge_new_indexation(df, file, columns=cols_to_merge)
    col_name = str("INDEX_" + name)
    col_name_unimarc = str("INDEX_UNI_" + name)
    df.rename(columns={"INDEXATION AFFICHAGE PUBLIC": col_name}, inplace=True)
    df.rename(columns={"INDEXATION UNIMARC": col_name_unimarc}, inplace=True)
    print("Dimension after merge: ", df.shape)
    index_columns.append(col_name)


Dealing with : AFE
Dimension after merge:  (100, 6)
Dealing with : MCR
Dimension after merge:  (100, 8)
Dealing with : JMF
Dimension after merge:  (112, 10)
Dealing with : LPL
Dimension after merge:  (112, 12)
Dealing with : LJZ
Dimension after merge:  (112, 14)
Dealing with : MPD
Dimension after merge:  (112, 16)


In [19]:
# Export csv of merged files
df.to_csv("fusion_fichiers_reindexation.csv", sep="\t", encoding="utf-8")
df.to_excel("fusion_fichiers_reindexation.xlsx")

# Check concepts

In [20]:
ram

,PPN,NOM
0,157992527,Kirp?n
1,110140494,Militaires artistes
2,028492161,Militaires romains
3,028521757,Militaires prussiens
4,029895561,Sa-skya-pa
...,...,...
85871,191405949,Relations artistes-collectivité
85872,234777605,Postcolonialisme et arts
85873,193907445,Ascèse
85874,027415074,Vie chrétienne


In [21]:
# Mettre à plat TOUS les mots clé
pattern = r';;\s*(?![^()$]*\))| -- |-- | --|--| - '

for col in index_columns:
    name = col.split("_")[1]
    print("Dealing with: ", name)
    new_col = str('rameau_concept_' + name)
    df[new_col] = df[col].apply(lambda x: re.split(pattern, str(x)))


Dealing with:  AFE
Dealing with:  MCR
Dealing with:  JMF
Dealing with:  LPL
Dealing with:  LJZ
Dealing with:  MPD


In [22]:
df.head(3)

,N°ordre,PPN,TITRE,RESUME,INDEX_UNI_AFE,INDEX_AFE,INDEX_UNI_MCR,INDEX_MCR,INDEX_UNI_JMF,INDEX_JMF,...,INDEX_UNI_LJZ,INDEX_LJZ,INDEX_UNI_MPD,INDEX_MPD,rameau_concept_AFE,rameau_concept_MCR,rameau_concept_JMF,rameau_concept_LPL,rameau_concept_LJZ,rameau_concept_MPD
0,79.0,308838,Les sommets de l'État : essai sur l'élite du p...,"Du XIXe siècle à nos jours, l'Etat ""fort"" à la...",606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France -- Histoire;;Pou...,606 $3027365581$aPouvoir (sciences sociales)$3...,Pouvoir (sciences sociales) -- Classes dirigea...,NaN,NaN,...,606$3027223345$aClasses dirigeantes$3027226794...,Classes dirigeantes -- France;;Hauts fonctionn...,NaN,NaN,"[Classes dirigeantes, France, Histoire, Pouvoi...","[Pouvoir (sciences sociales), Classes dirigean...",[nan],"[Hauts fonctionnaires, France, Classes dirigea...","[Classes dirigeantes, France, Hauts fonctionna...",[nan]
1,58.0,3632806,Les intellectuels sous la Ve République : 1958...,"Célèbres, influents, on les voit, on en parle ...",606$3027235149$aIntellectuels$3027284948$yFran...,Intellectuels -- France -- 1958-.... (5e Répub...,606 $3027235149$aIntellectuels$3027226794$yFra...,Intellectuels -- France -- 1945,NaN,NaN,...,606$3027235149$aIntellectuels$3027226794$yFran...,Intellectuels -- France;;Vie intellectuelle --...,NaN,NaN,"[Intellectuels, France, 1958-.... (5e Républiq...","[Intellectuels, France, 1945]",[nan],"[Intellectuels, France, 1958 (5e République), ...","[Intellectuels, France, Vie intellectuelle, Fr...",[nan]
2,59.0,47450037,"Bouddha, bouddhisme","La 4e de couv. indique : ""Ce petit livre répon...",606$3027321835$aBouddhisme,Bouddhisme,606 $3027321835$aBouddhisme,Bouddhisme,NaN,NaN,...,606$3027321835$aBouddhisme$3030974208$xCoutume...,Bouddhisme -- Coutumes et pratiques;;Bouddhism...,NaN,NaN,[Bouddhisme],[Bouddhisme],[nan],"[Bouddhisme, Etude et enseignement, Philosophi...","[Bouddhisme, Coutumes et pratiques, Bouddhisme...",[nan]


## Format data

In [23]:
df.columns

Index(['N°ordre', 'PPN', 'TITRE', 'RESUME', 'INDEX_UNI_AFE', 'INDEX_AFE',
       'INDEX_UNI_MCR', 'INDEX_MCR', 'INDEX_UNI_JMF', 'INDEX_JMF',
       'INDEX_UNI_LPL', 'INDEX_LPL', 'INDEX_UNI_LJZ', 'INDEX_LJZ',
       'INDEX_UNI_MPD', 'INDEX_MPD', 'rameau_concept_AFE',
       'rameau_concept_MCR', 'rameau_concept_JMF', 'rameau_concept_LPL',
       'rameau_concept_LJZ', 'rameau_concept_MPD'],
      dtype='object')

In [25]:
col_to_keep = ["PPN", "rameau_concept_AFE",
               "rameau_concept_MCR", "rameau_concept_JMF",
                "rameau_concept_LPL", "rameau_concept_LJZ", "rameau_concept_MPD"]
df_pivot = pd.melt(df[col_to_keep], id_vars=["PPN"])
df_pivot

,PPN,variable,value
0,308838,rameau_concept_AFE,"[Classes dirigeantes, France, Histoire, Pouvoi..."
1,3632806,rameau_concept_AFE,"[Intellectuels, France, 1958-.... (5e Républiq..."
2,47450037,rameau_concept_AFE,[Bouddhisme]
3,58296182,rameau_concept_AFE,[Philosophie et sciences]
4,59911174,rameau_concept_AFE,"[Culte impérial, Rome (Italie), Empereurs, Rom..."
...,...,...,...
667,076503909,rameau_concept_MPD,"[Energie éolienne en mer, Droit, Eoliennes, Lo..."
668,076986152,rameau_concept_MPD,"[Autonomie et mouvements indépendantistes, Nou..."
669,077463560,rameau_concept_MPD,[Francophonie]
670,077880560,rameau_concept_MPD,"[Mélanome, Diagnostic, Facteur de croissance d..."


In [26]:
labels=df_pivot["value"]
labels

0      [Classes dirigeantes, France, Histoire, Pouvoi...
1      [Intellectuels, France, 1958-.... (5e Républiq...
2                                           [Bouddhisme]
3                              [Philosophie et sciences]
4      [Culte impérial, Rome (Italie), Empereurs, Rom...
                             ...                        
667    [Energie éolienne en mer, Droit, Eoliennes, Lo...
668    [Autonomie et mouvements indépendantistes, Nou...
669                                       [Francophonie]
670    [Mélanome, Diagnostic, Facteur de croissance d...
671    [Astronomie, Histoire, Astronomes, Système sol...
Name: value, Length: 672, dtype: object

In [27]:
def flatten(list):
    flat_list = [item for sublist in list for item in sublist]
    return flat_list

In [28]:
keywords = flatten(labels)
print(f"Le dataset contient {len(set(keywords))} concepts RAMEAU differents")

Le dataset contient 925 concepts RAMEAU differents


In [30]:
all_concepts = df["rameau_concept_AFE"] + df["rameau_concept_MCR"] + df["rameau_concept_JMF"] + df["rameau_concept_LPL"] + df["rameau_concept_LJZ"]	+ df["rameau_concept_MPD"]
keywords2 = flatten(all_concepts)
print(f"Le dataset contient {len(set(keywords2))} concepts RAMEAU differents")

Le dataset contient 925 concepts RAMEAU differents


In [31]:
# Verification que toutes les vedettes appartiennent au referenciel RAMEAU
RAMEAU = ram["NOM"].to_list()
keywords = list(map(lambda x: x.lstrip().rstrip(), keywords)) 
if all([item in RAMEAU for item in keywords]):
    print("Tous les mots clés sont des concepts rameau")
else:
    check = [item in RAMEAU for item in keywords]
    bad_keywords = []
    for i in range(len(check)):
        if not check[i]:
            bad_keywords.append(keywords[i])
 
    unknown_concepts = list(set(bad_keywords))
    print(f"{len(unknown_concepts)} mots clés absents du répertoire RAMEAU sur un total de {len(set(keywords))}")

229 mots clés absents du répertoire RAMEAU sur un total de 866


In [32]:
# Save list of badly formatted concepts, for check
with open(os.path.join(data_path, "concepts_a_verifier_reindexation.txt"), "w") as output:
    output.write(str(unknown_concepts))
unknown_concepts

['',
 'Jurisprudence',
 'Néolitihique ancien',
 'Jusrisprudence',
 'Colonialisme et ethnologie',
 'Patrons (couture)',
 "Economie de l'environnement",
 'Suisse',
 'Aménagement végétal',
 'Ecoterrorisme',
 'Egypte',
 'Ethologie',
 'Scandinavie',
 'Extémisme (politique)',
 'Pays de langue française',
 'Centres médico-psychopédagogiques',
 'Brevet Informatique et Internet (pédagogie)',
 'Anecdotes',
 'Aspect religieux',
 'Afrique francophone',
 'Littérature africaine',
 "Professeurs d'éducation",
 'Algortithmes',
 'Récits de voyages',
 '2000-....',
 '1789-1799 (Révolution)',
 'Guerre de Vendée',
 'Grande-Bretagne',
 'Doctrines',
 '1500-1800',
 'Afrique',
 'Conflit isréalo-arabe',
 '1945-1970',
 'Escrocs et escroquerie',
 'Littérature française pour la jeunesse',
 'Méthodes pédagogiques',
 '1970-….',
 'Energie nucléaire',
 'Procéudure (droit européen)',
 'Innovations technologiques',
 'Ecrivains africains de la langue française',
 'France',
 'Art et Sciences',
 '1945',
 'Chansons',
 'Contr